In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

In [2]:
def calc_k(k,b):
    return np.sum(b[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]*k,axis=0)

In [303]:
def heuns(tn,x,function,h):
    k1 = h*function(tn,x)
    k2 = h*function(tn+h,x+k1)
    return (k1+k2)/2

In [302]:
def merson(tn,x,function,h):
    k1 = h*function(tn,x)
    k2 = h*function(tn+h/3,x+k1/3)
    k3 = h*function(tn+h/3,x+(k1+k2)/6)
    k4 = h*function(tn+h/2,x+(k1+k3*3)/8)
    k5 = h*function(tn+h,x+k1/2-3*k3/2+2*k4)
    return k1/6+2*k4/3+k5/6

In [304]:
pos = np.mgrid[0:5,0:5]*10.
pos[0,2,2] = 25.
vel = np.zeros_like(pos)
state = np.array([pos,vel])
import pandas as pd

In [429]:
spring_len = lambda direction,ballz: np.sqrt(np.sum((direction-ballz)**2,axis = 0))
def simple_update(tn,x):
    mass = x[0,:,1:-1,1:-1]
    north = x[0,:,0:-2,1:-1]
    south = x[0,:,2:,1:-1]
    west = x[0,:,1:-1,0:-2]
    east = x[0,:,1:-1,2:]
    
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:-1,1:-1] = -(k/m)*(update_north + update_south + \
                              update_east + update_west)

    result[0] = x[1]
    return result

In [430]:
k = 30
L = 10
g = 9.81
m = 10

In [489]:
def spring_field_rk( update_function, step = 0.01, filename = None):
    import matplotlib.animation as animation
    global state

    
    pos = np.mgrid[0:7,0:5]*10.
    vel = np.zeros_like(pos)
    pos[0,2,2] = 25.
    pos[1,2,2] = 25.
    state = np.array([pos,vel])
    
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(np.min(state[0,0])-20,np.max(state[0,0])+20)
    plt.xlim(np.min(state[0,1])-20,np.max(state[0,1])+20)

    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    v_springs = []
    for i in range(state.shape[2]):
        v_springs.append(ax.plot(state[0,1,i,:],state[0,0,i,:], linestyle='--', color='red')[0])
        
    h_springs = []
    for i in range(state.shape[3]):
        h_springs.append(ax.plot(state[0,1,:,i],state[0,0,:,i], linestyle='--', color = 'red')[0])
    
        
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    
    
    def init():
        
        return None
    
    def animate(t):
        global state
        
        state += merson(tn=t*step,x=state,function=update_function,h=step)
        
        points.set_data(state[0,1].ravel(),state[0,0].ravel())
        
        time_text.set_text(time_template % (t*step))
        
        for i in range(state.shape[2]):
            v_springs[i].set_data(state[0,1,i,:],state[0,0,i,:])
            
        for i in range(state.shape[3]):
            h_springs[i].set_data(state[0,1,:,i],state[0,0,:,i])
        
        return None
    
    ani = animation.FuncAnimation(fig, animate, np.arange(1000),
                              interval=1000*step, blit=False, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [491]:
spring_field_rk(update_function=simple_update, filename='spring_field_1a')

In [480]:
c = 0.1

In [481]:
def resistance_update(tn,x):
    mass = x[0,:,1:-1,1:-1]
    north = x[0,:,0:-2,1:-1]
    south = x[0,:,2:,1:-1]
    west = x[0,:,1:-1,0:-2]
    east = x[0,:,1:-1,2:]
    
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:-1,1:-1] = -(k/m)*(update_north + update_south + \
                              update_east + update_west) - \
                            np.sign(x[1,:,1:-1,1:-1])*c*x[1,:,1:-1,1:-1]**2 

    result[0] = x[1]
    return result

In [482]:
spring_field_rk(update_function=resistance_update)

In [463]:
def resistance_gravity_update(tn,x):
    mass = x[0,:,1:-1,1:-1]
    north = x[0,:,0:-2,1:-1]
    south = x[0,:,2:,1:-1]
    west = x[0,:,1:-1,0:-2]
    east = x[0,:,1:-1,2:]
    
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:-1,1:-1] = -(k/m)*(update_north + update_south + \
                              update_east + update_west) - \
                            np.sign(x[1,:,1:-1,1:-1])*c*x[1,:,1:-1,1:-1]**2
    
    result[1,0,1:-1,1:-1] -= g

    result[0] = x[1]
    return result

In [484]:
spring_field_rk(update_function=resistance_gravity_update)